In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [59]:
# Inicializa el detector ORB
orb = cv2.ORB_create()

# Inicializa el matcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

def compute_matches(detector, descriptor_matcher, img1, img2):
    # Encuentra los puntos clave y los descriptores con ORB
    kp1, des1 = detector.detectAndCompute(img1, None)
    kp2, des2 = detector.detectAndCompute(img2, None)

    # Empareja los descriptores
    matches = descriptor_matcher.match(des1, des2)

    # Filtra los emparejamientos con el test de ratio de Lowe
    matches = sorted(matches, key=lambda x: x.distance)
    good_matches = matches[:int(len(matches) * 0.15)]  # Conserva solo el 15% de los mejores emparejamientos

    return kp1, kp2, good_matches

def stitch_images(img1, img2, kp1, kp2, matches):
    # Extrae las ubicaciones de los puntos emparejados para la homografía
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Calcula la matriz de homografía
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Usa la homografía para transformar las esquinas de img1 a img2
    h, w = img2.shape[:2]
    pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, H)

    # Encuentra los límites para la imagen final
    [x_min, y_min] = np.int32(dst.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(dst.max(axis=0).ravel() + 0.5)
    translation_dist = [-x_min, -y_min]

    # Tamaño de la imagen de salida
    H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])

    # Warp de la imagen de salida
    output_img = cv2.warpPerspective(img1, H_translation.dot(H), (x_max - x_min, y_max - y_min))
    output_img[translation_dist[1]:h + translation_dist[1], translation_dist[0]:w + translation_dist[0]] = img2

    return output_img


In [60]:
# Nombres de archivo de las imágenes de entrada
filenames = ['IMG_IZQ.JPG', 'IMG_MID.JPG', 'IMG_DER.JPG']
H_filenames = ['H_IZQ.jpeg', 'H_MID.jpeg', 'H_DER.jpeg']
V_filenames = ['V_IZQ.jpeg', 'V_MID.jpeg', 'V_DER.jpeg']
profe_filenames = ['img1.png','img2.png','img3.png']

images = [cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in H_filenames]  # Asegúrate de cargarlas en escala de grises

# Calcula los emparejamientos
kp_left, kp_central, matches_left_central = compute_matches(orb, bf, images[0], images[1])
kp_right, kp_central, matches_right_central = compute_matches(orb, bf, images[2], images[1])

# Une la imagen izquierda a la central
left_to_central = stitch_images(images[0], images[1], kp_left, kp_central, matches_left_central)
plt.imshow(cv2.cvtColor(left_to_central, cv2.COLOR_BGR2RGB))

# Une la imagen derecha a la central
right_to_central = stitch_images(images[2], images[1], kp_right, kp_central, matches_right_central)
plt.imshow(cv2.cvtColor(right_to_central, cv2.COLOR_BGR2RGB))


ValueError: could not broadcast input array from shape (1600,1200) into shape (1600,650)